## Tilpasning af tokenizers

- De fleste analyser af tekst vil kræve en form for tokenization
- Simpel tokenization er hurtigt, men giver meget "støj"
- Brug af sprogmodeller i tokenization kan gøre processen mere præcis, men gør det typisk også langsommere

In [9]:
import pandas as pd 
import spacy
from spacy.lang.da import Danish
import re

tweetdata_url = "https://raw.githubusercontent.com/CALDISS-AAU/course_ndms-I/master/datasets/poltweets_sample.csv"
tweets_df = pd.read_csv(tweetdata_url)
tweets_df.head()

,created_at,id,full_text,is_quote_status,retweet_count,favorite_count,favorited,retweeted,is_retweet,hashtags,urls,user_followers_count,party
0,2020-10-21 14:48:39+00:00,1318927184111730700,Er på vej i miljøministeriet for at foreslå at...,False,13,47,False,False,False,['dkgreen'],[],4064,Alternativet
1,2019-06-02 20:03:20+00:00,1135275725592891400,@nielscallesoe @helenehagel @alternativet_ Det...,False,0,1,False,False,False,[],[],4064,Alternativet
2,2016-03-10 09:07:52+00:00,707855478320189400,"Vi står sammen, smiler Løkke på KL-topmøde og ...",False,13,14,False,False,False,"['dkpol', 'KLtop16']",[],4064,Alternativet
3,2019-04-07 19:59:03+00:00,1114980930467315700,@AnnaBylov @EU_Spring @rasmusnordqvist 💚,False,0,2,False,False,False,[],[],4064,Alternativet
4,2017-05-28 09:59:26+00:00,868768670427828200,Der er ikke noget alternativ til at Alternativ...,False,6,28,False,False,False,['LMÅ17'],"[{'url': 'https://t.co/3MCdZZGKRq', 'expanded_...",4064,Alternativet


In [18]:
nlp = Danish() # Indlæser "tom" sprogmodel
tokenizer = nlp.tokenizer # henter tokenizer

def tokenizer_simple(text):
    custom_stop_words = ["dkpol", "...", "\n\n", "tak", "når", "bare", "ja"]
    stop_words = list(nlp.Defaults.stop_words) + custom_stop_words
    
    
    doc = tokenizer(text)
    
    tokens = []
    
    for word in doc:
        if len(word.text) < 2:
            continue
        if word.text.lower() not in stop_words:
            tokens.append(word.text.lower())
    
    return(tokens)

In [19]:
tweets_df['tokens'] = tweets_df['full_text'].apply(tokenizer_simple)
tweets_df_tidy = tweets_df.explode('tokens')
tweets_df_tidy['tokens'].value_counts()[0:10]

@alternativet    333
godt             291
danmark          206
dag              196
helt             185
nye              172
regeringen       165
nej              165
@spolitik        157
dkgreen          155
Name: tokens, dtype: int64

In [13]:
tweet = tweets_df.loc[1505, 'full_text']
tokens = tokenizer_simple(tweet)

print(tweet,
     "\n",
     tokens)

God udstrakt hånd med grønt råderum på 2 mia. kr. om året. Det er en bevægelse i den rigtige retning efter stort politisk pres. Det viser og udstiller desuden, at regeringens egne grønne ambitioner i det fremlagte forslag for #FL21 var og er helt utilstrækkeligt. #ftlive#dkpol 
 ['God', 'udstrakt', 'hånd', 'grønt', 'råderum', '2', 'mia.', 'kr.', 'året', '.', 'Det', 'bevægelse', 'rigtige', 'retning', 'stort', 'politisk', 'pres', '.', 'Det', 'viser', 'udstiller', 'desuden', ',', 'regeringens', 'egne', 'grønne', 'ambitioner', 'fremlagte', 'forslag', '#', 'FL21', 'helt', 'utilstrækkeligt', '.', '#', 'ftlive#dkpol']


In [27]:
nlp = spacy.load("da_core_news_sm", disable = ["ner"])

In [31]:
doc = nlp(tweet)

for word in doc:
    print(word.text, "\t", word.lemma_)

God 	 God
udstrakt 	 udstrække
hånd 	 hånd
med 	 mede
grønt 	 grønt
råderum 	 råderum
på 	 på
2 	 2
mia. 	 mia.
kr. 	 kr.
om 	 om
året 	 år
. 	 .
Det 	 Det
er 	 være
en 	 en
bevægelse 	 bevægelse
i 	 i
den 	 den
rigtige 	 rigtig
retning 	 retning
efter 	 efter
stort 	 stor
politisk 	 politisk
pres 	 presse
. 	 .
Det 	 Det
viser 	 vise
og 	 og
udstiller 	 udstille
desuden 	 desuden
, 	 ,
at 	 at
regeringens 	 regering
egne 	 egn
grønne 	 grøn
ambitioner 	 ambition
i 	 i
det 	 det
fremlagte 	 fremlægge
forslag 	 forslag
for 	 for
# 	 #
FL21 	 FL21
var 	 være
og 	 og
er 	 være
helt 	 helt
utilstrækkeligt 	 utilstrækkelig
. 	 .
# 	 #
ftlive#dkpol 	 ftlive#dkpol


In [36]:
default_prefixes = list(nlp.Defaults.prefixes)
default_prefixes.remove("#")
prefix_regex = spacy.util.compile_prefix_regex(default_prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [39]:
def tokenizer_spacy(text):
    custom_stop_words = ["dkpol", "...", "\n\n", "tak", "når", "bare", "ja"]
    stop_words = list(nlp.Defaults.stop_words) + custom_stop_words
    
    doc = nlp(text)
    
    pos_tags = ["NOUN", "ADJ"]
    
    tokens = []
    
    for word in doc:
        if word.text.startswith("@"):
            continue
        if word.text.startswith("#"):
            continue
        if len(word.text) < 2:
            continue
        if word.text.lower() not in stop_words and word.pos_ in pos_tags:
            tokens.append(word.lemma_.lower())
            
    return(tokens)

In [40]:
tweets_df['tokens'] = tweets_df['full_text'].apply(tokenizer_spacy)
tweets_df_tidy = tweets_df.explode('tokens')
tweets_df_tidy['tokens'].value_counts()[0:10]

dag         280
regering    228
stor        198
dansk       193
år          189
barn        164
god         148
gang        141
penge       125
vigtig      120
Name: tokens, dtype: int64

In [41]:
tweets_df_tidy['tokens'].value_counts()[0:20]

dag         280
regering    228
stor        198
dansk       193
år          189
barn        164
god         148
gang        141
penge       125
vigtig      120
grøn        117
tid         116
ny          110
enig        108
dansker     106
hele         99
gode         98
forslag      95
parti        94
valg         94
Name: tokens, dtype: int64